In [1]:
import sys
sys.path.append('../')
import os
os.chdir('../')
%reload_ext autoreload
%autoreload 2

In [2]:
import os 
import torch
import numpy as np

from torch.utils.tensorboard import SummaryWriter
from os.path import join as pjoin
from torch.distributions import Categorical
import json
import clip

import options.option_transformer as option_trans
import models.vqvae as vqvae
import utils.utils_model as utils_model
import utils.eval_trans as eval_trans
from dataset import dataset_TM_train
from dataset import dataset_TM_eval
from dataset import dataset_tokenize
import models.t2m_trans as trans
from options.get_eval_option import get_opt
from models.evaluator_wrapper import EvaluatorModelWrapper
import warnings
warnings.filterwarnings('ignore')
from exit.utils import get_model, visualize_2motions, generate_src_mask, uniform, cosine_schedule
from einops import rearrange, repeat
import os
import torch.nn.functional as F
# os.environ["CUDA_LAUNCH_BLOCKING"]="1"
# os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [3]:
class Temp:
    def __init__(self):
        print('mock:: opt')
args = Temp()
args.dataname = args.dataset_name = 'kit'
args.down_t = 2

train_loader_token = dataset_tokenize.DATALoader(args.dataname, 1, unit_length=2**args.down_t)

from utils.word_vectorizer import WordVectorizer
w_vectorizer = WordVectorizer('./glove', 'our_vab')
val_loader = dataset_TM_eval.DATALoader(args.dataname, False, 32, w_vectorizer)

dataset_opt_path = 'checkpoints/kit/Comp_v6_KLD005/opt.txt' if args.dataname == 'kit' else 'checkpoints/t2m/Comp_v6_KLD005/opt.txt'

wrapper_opt = get_opt(dataset_opt_path, torch.device('cuda'))
eval_wrapper = EvaluatorModelWrapper(wrapper_opt)

mock:: opt


100%|██████████| 300/300 [00:00<00:00, 3071.22it/s]


Pointer Pointing at 0
Reading checkpoints/kit/Comp_v6_KLD005/opt.txt
Loading Evaluation Model Wrapper (Epoch 30) Completed!!


In [4]:
##### ---- Network ---- #####
clip_model, clip_preprocess = clip.load("ViT-B/32", device=torch.device('cuda'), jit=False)  # Must set jit=False for training
clip.model.convert_weights(clip_model)  # Actually this line is unnecessary since clip by default already on float16
clip_model.eval()
for p in clip_model.parameters():
    p.requires_grad = False

# https://github.com/openai/CLIP/issues/111
class TextCLIP(torch.nn.Module):
    def __init__(self, model) :
        super(TextCLIP, self).__init__()
        self.model = model
        
    def forward(self,text):
        return self.model.encode_text(text)
clip_model = TextCLIP(clip_model)

In [5]:
args.nb_code = 512 # 8192 # 
args.code_dim = 512 # 32 # 
args.output_emb_width = 512
args.down_t = 2
args.stride_t = 2
args.width = 512
args.depth = 3
args.dilation_growth_rate = 3
args.vq_act = 'relu'
args.vq_norm = None
args.quantizer = 'ema_reset'
args.mu = 0.99
args.embed_dim_gpt = 1024
args.clip_dim = 512
args.block_size = 51+1
args.num_layers = 9
args.n_head_gpt = 16
args.drop_out_rate = 0.1
args.ff_rate = 4

if args.dataset_name == 'kit':
    args.vq_name = 'VQVAE'
elif args.dataset_name == 't2m':
    args.vq_name = '2_VQVAE_HML3D'
args.exp_name = 'TEMP'
args.out_dir = 'output_GPT_Final'
args.out_dir = os.path.join(args.out_dir, f'{args.exp_name}')
args.vq_dir= f'output/{args.vq_name}'
codebook_dir = f'{args.vq_dir}/codebook/'
os.makedirs(args.out_dir, exist_ok = True)
os.makedirs(args.vq_dir, exist_ok = True)
os.makedirs(codebook_dir, exist_ok = True)

net = vqvae.HumanVQVAE(args, ## use args to define different parameters in different quantizers
                       args.nb_code,
                       args.code_dim,
                       args.output_emb_width,
                       args.down_t,
                       args.stride_t,
                       args.width,
                       args.depth,
                       args.dilation_growth_rate)
trans_encoder = trans.Text2Motion_Transformer(num_vq=args.nb_code, 
                                    embed_dim=args.embed_dim_gpt, 
                                    clip_dim=args.clip_dim, 
                                    block_size=args.block_size, 
                                    num_layers=args.num_layers, 
                                    n_head=args.n_head_gpt, 
                                    drop_out_rate=args.drop_out_rate, 
                                    fc_rate=args.ff_rate)
args.resume_pth = f'/home/epinyoan/git/MaskText2Motion/T2M-BD/output/{args.vq_name}/net_last.pth'
ckpt = torch.load(args.resume_pth, map_location='cpu')
net.load_state_dict(ckpt['net'], strict=True)
# net = torch.nn.DataParallel(net)

# args.resume_trans = '/home/epinyoan/git/MaskText2Motion/T2M-BD/output/TEMP/net_last.pth'
# ckpt = torch.load(args.resume_trans, map_location='cpu')
# trans_encoder.load_state_dict(ckpt['trans'], strict=True)
trans_encoder = torch.nn.DataParallel(trans_encoder)
net.eval()
net.cuda()
trans_encoder.train()
trans_encoder.cuda()
''

''

### Train

In [6]:
args.batch_size = 2
num_workers = 8
train_loader = dataset_TM_train.DATALoader(args.dataname, args.batch_size, args.nb_code, codebook_dir, unit_length=2**args.down_t, num_workers=num_workers)
train_loader_iter = dataset_TM_train.cycle(train_loader)

100%|██████████| 4888/4888 [00:01<00:00, 2742.52it/s]


In [7]:
args.pkeep = .5
batch = next(train_loader_iter)
clip_text, target, m_tokens_len = batch
target, m_tokens_len = target.cuda(), m_tokens_len.cuda()
bs = target.shape[0]

text = clip.tokenize(clip_text, truncate=True).cuda()
    
feat_clip_text = clip_model(text).float()

# [INFO] Swap input index
if args.pkeep == -1:
    proba = np.random.rand(1)[0]
    mask = torch.bernoulli(proba * torch.ones(target.shape,
                                                        device=target.device))
else:
    mask = torch.bernoulli(args.pkeep * torch.ones(target.shape,
                                                        device=target.device))
mask = mask.round().to(dtype=torch.int64)
r_indices = torch.randint_like(target, args.nb_code)
input_indices = mask*target+(1-mask)*r_indices


In [8]:
# Time step masking
batch_size, max_len = target.shape[:2]
mask_id = get_model(net).vqvae.num_code + 2
rand_time = uniform((batch_size,), device = target.device)
rand_mask_probs = cosine_schedule(rand_time)
num_token_masked = (m_tokens_len * rand_mask_probs).round().clamp(min = 1)

# batch_randperm = torch.rand((batch_size, max_len), device = target.device).argsort(dim = -1)
seq_mask = generate_src_mask(max_len, m_tokens_len+1)
batch_randperm = torch.rand((batch_size, max_len), device = target.device) - seq_mask.int()
batch_randperm = batch_randperm.argsort(dim = -1)
mask_token = batch_randperm < rearrange(num_token_masked, 'b -> b 1')

masked_target = torch.where(mask_token, input=target, other=-1)
masked_input_indices = torch.where(mask_token, mask_id, input_indices)

cls_pred = trans_encoder(masked_input_indices, feat_clip_text, seq_mask)[:, 1:]
cls_pred = cls_pred.contiguous()

# check that the random applies only actual seq not the blank
torch.equal((mask_token * seq_mask).sum(-1), mask_token.sum(-1))

True

In [9]:
### TEST transformer
feat_clip_text = torch.rand([2, 512])
masked_input_indices = torch.ones([2, 51]).int()
index_motion = trans_encoder(masked_input_indices, feat_clip_text, src_mask=seq_mask)

In [11]:
# 4. xent and apply weight later
loss_cls4 = F.cross_entropy(rearrange(cls_pred, 'b n c -> b c n'), masked_target, reduction = 'none')
weights = mask_token / (mask_token.sum(-1).unsqueeze(-1) * mask_token.shape[0])
loss_cls4 = (loss_cls4*weights).sum()
loss_cls4, weights.sum()

# 5. mask first and xent later. Since we need "cls_pred_all_masked" to calculate acc so use this version
cls_pred_all_masked = torch.masked_select(cls_pred, mask_token.unsqueeze(-1)).view(-1, cls_pred.shape[-1])
target_all_masked = torch.masked_select(target, mask_token)
weight_all_masked = torch.masked_select(weights, mask_token)
loss_cls5 = F.cross_entropy(cls_pred_all_masked, target_all_masked, reduction = 'none')
loss_cls5 = (loss_cls5 * weight_all_masked).sum()
loss_cls4, loss_cls5

(tensor(6.4958, device='cuda:0', grad_fn=<SumBackward0>),
 tensor(6.4958, device='cuda:0', grad_fn=<SumBackward0>))

In [41]:
######### Compute Acc. #########
args.if_maxtest = True # [INFO] for deterministic testing
probs = torch.softmax(cls_pred_all_masked, dim=-1)
if args.if_maxtest:
    _, cls_pred_index = torch.max(probs, dim=-1)
else:
    dist = Categorical(probs)
    cls_pred_index = dist.sample()
(cls_pred_index == target_all_masked).sum().item(), 
# cls_pred_index, target_all_masked

(0,)

In [16]:
###### Loss without random mask 

###### Compute Batch Xent ######
loss_ce = torch.nn.CrossEntropyLoss(reduction='none')
cb_idx_mask = generate_src_mask(target.shape[1], m_tokens_len+1)
cls_pred_all_masked = torch.masked_select(cls_pred, cb_idx_mask.unsqueeze(-1)).view(-1, cls_pred.shape[-1])
target_all_masked = torch.masked_select(target, cb_idx_mask)

denom = torch.ones(*target.shape).cuda() * bs * (m_tokens_len+1).unsqueeze(-1)
denom = torch.masked_select(denom, cb_idx_mask)

loss_cls = loss_ce(cls_pred_all_masked, target_all_masked) / denom
loss_cls = loss_cls.sum()

##### 2nd batch method compute loss first select later but maybe not faster b/c need to compute more loss
# T2M-GPT was working but BD got error: "RuntimeError: CUDA error: device-side assert triggered"
##############################################################################
# denom = torch.ones(*target.shape).cuda() * bs * (m_tokens_len+1).unsqueeze(-1)
# print(cls_pred.shape, target.shape, denom.shape, cb_idx_mask.shape)
# cls_pred.view(-1, cls_pred.shape[-1])
# loss_cls2 = loss_ce(cls_pred.view(-1, cls_pred.shape[-1]), 
#                     target.view(-1)).view(*denom.shape)/denom
# loss_cls2 = torch.masked_select(loss_cls2, cb_idx_mask).sum()


##### 3rd compute xent bf weight #####
loss_cls3 = F.cross_entropy(rearrange(cls_pred, 'b n c -> b c n'), target, reduction = 'none')
weights = generate_src_mask(target.shape[1], m_tokens_len+1)
weights = cb_idx_mask/(m_tokens_len+1).unsqueeze(-1) / cb_idx_mask.shape[0]
loss_cls3 = (loss_cls3*weights).sum()
loss_cls, loss_cls3

(tensor(6.4831, device='cuda:0', grad_fn=<SumBackward0>),
 tensor(6.4831, device='cuda:0', grad_fn=<SumBackward0>))

### Eval

In [6]:

val_loader = dataset_TM_eval.DATALoader(args.dataname, False, 32, w_vectorizer)

100%|██████████| 300/300 [00:00<00:00, 2305.66it/s]

Pointer Pointing at 0


In [7]:
class LoggerWriterMock:
    def __init__(self):
        self.info
    def info(self, *args):
        print(*args)
    def add_scalar(self, *args):
        print(*args)
    def add_video(self, *args):
        print(*args)
logger = LoggerWriterMock()
logger.info('test')
writer = LoggerWriterMock()
writer.add_scalar('./Test/FID', 'fid', 'nb_iter')

test
./Test/FID fid nb_iter


In [8]:
########## Mask Motion in Eval ####################
index_motion = torch.randint(0, 513, (8, 51)).cuda()
blank_id = 512 # get_model(trans_encoder).num_vq
index_motion[1, 5] = blank_id
index_motion[1, 7] = blank_id
index_motion[4, 9] = blank_id
index_motion[0, 0] = blank_id
index_motion[-1, 1] = blank_id

# [INFO] 1. this get the last index of blank_id
# pred_length = (index_motion == blank_id).int().argmax(1).float()
# [INFO] 2. this get the first index of blank_id
pred_length = (index_motion >= blank_id).int()
pred_length = torch.topk(pred_length, k=1, dim=1).indices.squeeze().float()
pred_length[pred_length==0] = index_motion.shape[1]

index_motion.shape, pred_length 

(torch.Size([8, 51]),
 tensor([51.,  5., 51., 51.,  9., 14., 51.,  1.], device='cuda:0'))

In [9]:
word_embeddings, pos_one_hots, clip_text, sent_len, pose, m_length, token, name = next(iter(val_loader))
seq = pose.shape[1]
seq_mask = generate_src_mask(get_model(trans_encoder).block_size-1, m_length+1)
feat_clip_text = torch.rand([32, 512])
index_motion = trans_encoder(feat_clip_text, type="sample", src_mask=seq_mask)

In [13]:
pred_pose_eval, pose, m_length, clip_text, best_fid, best_iter, best_div, best_top1, best_top2, best_top3, best_matching, writer, logger = \
    eval_trans.evaluation_transformer(args.out_dir, 
        val_loader, 
        net, 
        trans_encoder, 
        logger, 
        writer, 
        0, 
        best_fid=1000, 
        best_iter=0, 
        best_div=100, 
        best_top1=0, 
        best_top2=0, 
        best_top3=0, 
        best_matching=100, 
        clip_model=clip_model, 
        eval_wrapper=eval_wrapper)

100%|██████████| 47/47 [00:51<00:00,  1.09s/it]


--> 	 Eva. Iter 0 :, FID. 80.6462, Diversity Real. 9.5556, Diversity. 0.0000, R_precision_real. [0.51928191 0.71143617 0.81050532], R_precision. [0.03125 0.0625  0.09375], matching_score_real. 2.844394425128369, matching_score_pred. 8.74730614398388
./Test/FID 80.64619437323736 0
./Test/Diversity 0.0 0
./Test/top1 0.03125 0
./Test/top2 0.0625 0
./Test/top3 0.09375 0
./Test/matching_score 8.74730614398388 0
--> --> 	 FID Improved from 1000.00000 to 80.64619 !!!
--> --> 	 matching_score Improved from 100.00000 to 8.74731 !!!
--> --> 	 Diversity Improved from 100.00000 to 0.00000 !!!
--> --> 	 Top1 Improved from 0.0000 to 0.0312 !!!
--> --> 	 Top2 Improved from 0.0000 to 0.0625 !!!
--> --> 	 Top3 Improved from 0.0000 to 0.0938 !!!


In [18]:
i = 0
x = pose[i].detach().cpu().numpy()
y = pred_pose_eval[i].detach().cpu().numpy()
l = m_length[i]
caption = clip_text[i]
visualize_2motions(x, val_loader.dataset.std, val_loader.dataset.mean, args.dataname, l, y, save_path=None)